# Quickstart Guide

### Imports

In [ ]:
import pm4py
import utils.extractor as extractor
import utils.frames as frames
import math
import visualization.viz as viz
from simulation.simulator import Simulator
from simulation.objects.enums import Callbacks, SimulationModes, TimestampModes, OptimizationModes, SchedulingBehaviour
import utils.fairness as Fairness
import utils.congestion as Congestion
import utils.optimization as Optimization

### Callbacks for the MIP-Scheduling

In [ ]:
def SimulatorFairness_Callback(activeTraces, completedTraces, LonelyResources, R, windows, currentWindow):
    return Fairness.FairnessBacklogFair_TIME(activeTraces, completedTraces, LonelyResources, R, windows, currentWindow, BACKLOG_N=scriptArgs.FairnessBacklogN)

In [ ]:
def SimulatorCongestion_Callback(activeTraces, completedTraces, LonelyResources, A, R, windows, currentWindow, simTime):
    segmentFreq, segmentTime, waitingTraces = Congestion.GetActiveSegments(activeTraces, simTime, SimulationModes.KNOWN_FUTURE)
    return Congestion.GetProgressByWaitingNumberInFrontOfActivity(A, segmentFreq, waitingTraces)

In [ ]:
def SimulatorWindowStartScheduling_Callback(activeTraces, A, P_AtoR, availableResources, simTime, windowDuration, fRatio, cRatio, optimizationMode):
    return Optimization.OptimizeActiveTraces(activeTraces, A, P_AtoR, availableResources, simTime, windowDuration, fRatio, cRatio, optimizationMode)

### Simulator Setup

In [ ]:
log = pm4py.read_xes('logs/log_ResReduced.xes')


windowNumber = 100 * math.ceil(math.sqrt(sum([len(trace) for trace in log])))

# Convert XES-Events into dict {'act', 'ts', 'res', 'single', 'cid'}
event_dict = extractor.event_dict(log, res_info=True)

windowWidth = frames.get_width_from_number(event_dict, windowNumber)
bucketId_borders_dict = frames.bucket_window_dict_by_width(event_dict, windowWidth)
eventsPerWindowDict, id_frame_mapping = frames.bucket_id_list_dict_by_width(event_dict, windowWidth) # WindowID: [List of EventIDs] , EventID: WindowID       

In [ ]:
sim = Simulator(event_dict, eventsPerWindowDict, bucketId_borders_dict, 
        simulationMode      = Callbacks.KNOWN_FUTURE,     #PREDICTED_FUTURE
        optimizationMode    = OptimizationModes.FAIRNESS, #CONGESTION, BOTH 
        schedulingBehaviour = SchedulingBehaviour.CLEAR_ASSIGNMENTS_EACH_WINDOW, #KEEP_ASSIGNMENTS
        endTimestampAttribute='ts', verbose=False)
    
sim.Register(Callbacks.WND_START_SCHEDULING, SimulatorWindowStartScheduling_Callback)
sim.Register(Callbacks.CALC_Fairness,        SimulatorFairness_Callback)
sim.Register(Callbacks.CALC_Congestion,      SimulatorCongestion_Callback)

In [ ]:
# Run the simulation
sim.Run()
sim.ExportSimulationLog('logs/jupyter_Quickstart.xes')